In [ ]:
import pyspark


In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Nome do aplicativo").getOrCreate()


In [ ]:
from pyspark.ml.feature import RFormula

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
churn = spark.read.csv("/home/cliente/download/Churn.csv", inferSchema=True, header=True, sep=";")

In [ ]:
churn.show(5)

+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|       0|            1|        1|             1|       10134888|     1|
|        608|    Spain|Female| 41|     1| 8380786|            1|        0|             1|       11254258|     0|
|        502|   France|Female| 42|     8| 1596608|            3|        1|             0|       11393157|     1|
|        699|   France|Female| 39|     1|       0|            2|        0|             0|        9382663|     0|
|        850|    Spain|Female| 43|     2|12551082|            1|        1|             1|         790841|     0|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+-------

In [ ]:
formula = RFormula(formula="Exited ~.", featuresCol="features", labelCol="label", handleInvalid="skip")

In [ ]:
churn_trans = formula.fit(churn).transform(churn).select("features","label")

In [ ]:
churn_trans.show(truncate=False)

+----------------------------------------------------------------+-----+
|features                                                        |label|
+----------------------------------------------------------------+-----+
|[619.0,1.0,0.0,0.0,42.0,2.0,0.0,1.0,1.0,1.0,1.0134888E7]        |1.0  |
|[608.0,0.0,0.0,0.0,41.0,1.0,8380786.0,1.0,0.0,1.0,1.1254258E7]  |0.0  |
|[502.0,1.0,0.0,0.0,42.0,8.0,1596608.0,3.0,1.0,0.0,1.1393157E7]  |1.0  |
|(11,[0,1,4,5,7,10],[699.0,1.0,39.0,1.0,2.0,9382663.0])          |0.0  |
|[850.0,0.0,0.0,0.0,43.0,2.0,1.2551082E7,1.0,1.0,1.0,790841.0]   |0.0  |
|[645.0,0.0,0.0,1.0,44.0,8.0,1.1375578E7,2.0,1.0,0.0,1.4975671E7]|1.0  |
|[822.0,1.0,0.0,1.0,50.0,7.0,0.0,2.0,1.0,1.0,100628.0]           |0.0  |
|[376.0,0.0,1.0,0.0,29.0,4.0,1.1504674E7,4.0,1.0,0.0,1.1934688E7]|1.0  |
|[501.0,1.0,0.0,1.0,44.0,4.0,1.4205107E7,2.0,0.0,1.0,749405.0]   |0.0  |
|[684.0,1.0,0.0,1.0,27.0,2.0,1.3460388E7,1.0,1.0,1.0,7172573.0]  |0.0  |
|[528.0,1.0,0.0,1.0,31.0,6.0,1.0201672E7,2.0,0.0,0.

In [ ]:
churnTreino, churnTeste = churn_trans.randomSplit([0.7,0.3])

In [ ]:
print(churnTreino.count())

7063


In [ ]:
print(churnTeste.count())

2937


In [ ]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

In [ ]:
modelo = dt.fit(churnTreino)

In [ ]:
previsao = modelo.transform(churnTeste)

In [ ]:
previsao.show()

+--------------------+-----+--------------+--------------------+----------+
|            features|label| rawPrediction|         probability|prediction|
+--------------------+-----+--------------+--------------------+----------+
|(11,[0,1,3,4,7,10...|  0.0|[4433.0,523.0]|[0.89447134786117...|       0.0|
|(11,[0,1,4,5,7,10...|  1.0|  [42.0,229.0]|[0.15498154981549...|       1.0|
|(11,[0,1,4,5,7,10...|  0.0|[4433.0,523.0]|[0.89447134786117...|       0.0|
|(11,[0,1,4,5,7,10...|  0.0|[4433.0,523.0]|[0.89447134786117...|       0.0|
|(11,[0,1,4,5,7,10...|  0.0|[4433.0,523.0]|[0.89447134786117...|       0.0|
|(11,[0,1,4,5,7,10...|  0.0|[4433.0,523.0]|[0.89447134786117...|       0.0|
|(11,[0,1,4,5,7,10...|  0.0|[4433.0,523.0]|[0.89447134786117...|       0.0|
|(11,[0,1,4,5,7,10...|  1.0| [126.0,118.0]|[0.51639344262295...|       0.0|
|(11,[0,1,4,5,7,10...|  0.0|[4433.0,523.0]|[0.89447134786117...|       0.0|
|(11,[0,1,4,5,7,10...|  1.0|[4433.0,523.0]|[0.89447134786117...|       0.0|
|(11,[0,1,4,

In [ ]:
avaliar = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label", metricName="areaUnderROC")

In [ ]:
areaunderROC = avaliar.evaluate(previsao)

In [ ]:
print(areaunderROC)

0.7013262868160732
